In [ ]:
# ============================================================
# EN3150 Assignment 03 – Task 1
# ============================================================

# We did the tasks as two divided parts; so, to see the outputs of these codes, please check our Github repo.
# Task 01: https://github.com/Neuro-Matrix-EN3150/Assignment-03-CNN-for-image-classification/blob/main/CNN.ipynb
# Task 02: https://github.com/Neuro-Matrix-EN3150/Assignment-03-CNN-for-image-classification/blob/main/Part2%20new.ipynb


# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import random, os

In [ ]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Combine training and test sets
X = np.concatenate([x_train, x_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Normalize and reshape
X = X.astype("float32") / 255.0
X = np.expand_dims(X, -1)

# Split 70/15/15
train_size = 0.70
val_size = 0.15 / (1 - train_size)

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, train_size=val_size, stratify=y_rem, random_state=42)

# One-hot encode
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

In [ ]:
def build_model(input_shape=(28,28,1), num_classes=10):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = build_model()
model.summary()

In [ ]:
initial_lr = 1e-3
opt = Adam(learning_rate=initial_lr)

model = build_model()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

history = model.fit(
    X_train, y_train_cat,
    epochs=20,
    batch_size=128,
    validation_data=(X_val, y_val_cat),
    callbacks=[reduce_lr],
    verbose=2
)

# Plot training & validation loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (Adam)')
plt.legend()
plt.show()

In [ ]:
def train_with_optimizer(opt, opt_name, epochs=20):
    tf.keras.backend.clear_session()
    model = build_model()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    hist = model.fit(X_train, y_train_cat, validation_data=(X_val, y_val_cat),
                     epochs=epochs, batch_size=128, verbose=2)
    test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=1)
    return {'name': opt_name, 'model': model, 'history': hist, 'test_acc': test_acc, 'y_pred': y_pred}

results = []
results.append(train_with_optimizer(Adam(learning_rate=1e-3), "Adam"))
results.append(train_with_optimizer(SGD(learning_rate=0.01), "SGD"))
results.append(train_with_optimizer(SGD(learning_rate=0.01, momentum=0.9), "SGD+Momentum"))

for r in results:
    print(f"{r['name']} Test Accuracy: {r['test_acc']:.4f}")

In [ ]:
for r in results:
    print("="*50)
    print(f"Optimizer: {r['name']}")
    print("Test Accuracy:", round(r['test_acc'], 4))
    print(classification_report(y_test, r['y_pred'], digits=4))

    cm = confusion_matrix(y_test, r['y_pred'])
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {r["name"]}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
best = results[0]  # e.g., Adam performed best
y_pred = best['y_pred']

print("Final Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision (macro):", precision_score(y_test, y_pred, average='macro'))
print("Recall (macro):", recall_score(y_test, y_pred, average='macro'))
print("F1-score (macro):", f1_score(y_test, y_pred, average='macro'))

In [ ]:
# ============================================================
# EN3150 Assignment 03 – Task 2: Compare with State-of-the-Art Networks
# ============================================================

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import pandas as pd
import json

# ============================================================
# 1. Load MNIST (UCI equivalent)
# ============================================================
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()
X_full = np.concatenate([x_train_full, x_test_full], axis=0)
y_full = np.concatenate([y_train_full, y_test_full], axis=0)

# Optional: use subset for Colab memory safety
subset_size = 15000       # use 10k–20k to fit RAM
X_full = X_full[:subset_size]
y_full = y_full[:subset_size]

# ============================================================
# 2. Split dataset 70/15/15
# ============================================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y_full, test_size=0.30, random_state=42, stratify=y_full
    )
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# ============================================================
# 3. Preprocessing on-the-fly (memory efficient)
# ============================================================
IMG_SIZE = 224
BATCH = 64

def preprocess_grayscale_to_rgb(x, y):
    x = tf.expand_dims(x, -1)                  # (28,28,1)
    x = tf.image.resize(x, [IMG_SIZE, IMG_SIZE])
    x = tf.image.grayscale_to_rgb(x)           # (224,224,3)
    x = tf.cast(x, tf.float32) / 255.0
    return x, y

train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train))
            .shuffle(10000)
            .map(preprocess_grayscale_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(BATCH)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (tf.data.Dataset.from_tensor_slices((X_val, y_val))
          .map(preprocess_grayscale_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
          .batch(BATCH)
          .prefetch(tf.data.AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test))
           .map(preprocess_grayscale_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(BATCH)
           .prefetch(tf.data.AUTOTUNE))

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))

In [ ]:
# ============================================================
# 4. Custom CNN Model
# ============================================================
def build_custom_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=10,
                     x1=32, x2=64, x3=256, d=0.5):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(x1, (3,3), activation='relu', padding='same')(inp)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(x2, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(x3, activation='relu')(x)
    x = layers.Dropout(d)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)

custom_model = build_custom_cnn()
custom_model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
custom_model.summary()

EPOCHS = 20
history_custom = custom_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# ============================================================
# 5. Prepare datasets for ResNet and VGG (preprocessing)
# ============================================================
def map_resnet(x, y):
    return resnet_preprocess(x * 255.0), y

def map_vgg(x, y):
    return vgg_preprocess(x * 255.0), y

train_ds_resnet = train_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)
val_ds_resnet   = val_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)
test_ds_resnet  = test_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)

train_ds_vgg = train_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)
val_ds_vgg   = val_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)
test_ds_vgg  = test_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)



In [ ]:
# ============================================================
# 6. ResNet50 Transfer Learning
# ============================================================
num_classes = 10
base_res = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_res.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_res(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
resnet_model = models.Model(inputs, outputs)

resnet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                     loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_resnet_head = resnet_model.fit(train_ds_resnet, validation_data=val_ds_resnet, epochs=8)

# Fine-tune
base_res.trainable = True
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                     loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_resnet_ft = resnet_model.fit(train_ds_resnet, validation_data=val_ds_resnet, epochs=12)

In [ ]:
# ============================================================
# 7. VGG16 Transfer Learning
# ============================================================
def map_resnet(x, y):
    return resnet_preprocess(x * 255.0), y

def map_vgg(x, y):
    return vgg_preprocess(x * 255.0), y

train_ds_resnet = train_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)
val_ds_resnet   = val_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)
test_ds_resnet  = test_ds.map(map_resnet).prefetch(tf.data.AUTOTUNE)

train_ds_vgg = train_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)
val_ds_vgg   = val_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)
test_ds_vgg  = test_ds.map(map_vgg).prefetch(tf.data.AUTOTUNE)
num_classes = 10
base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_vgg.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_vgg(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
vgg_model = models.Model(inputs, outputs)

vgg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_vgg_head = vgg_model.fit(train_ds_vgg, validation_data=val_ds_vgg, epochs=8)

base_vgg.trainable = True
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_vgg_ft = vgg_model.fit(train_ds_vgg, validation_data=val_ds_vgg, epochs=12)


In [ ]:
# ============================================================
# 8. Plot training/validation losses
# ============================================================

def plot_history(histories, titles):
    plt.figure(figsize=(12,5))
    for h, t in zip(histories, titles):
        plt.plot(h.history['loss'], label=f'{t} Train Loss')
        plt.plot(h.history['val_loss'], '--', label=f'{t} Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.title('Training and Validation Loss')
    plt.show()

# Note: history_custom is from the custom CNN model training
# history_resnet_head and history_resnet_ft are from ResNet50 training
# history_vgg_head and history_vgg_ft are from VGG16 training

# Plot losses for Custom CNN
if 'history_custom' in locals() and history_custom is not None:
    plot_history([history_custom], ['Custom CNN'])

# Plot losses for ResNet50 (fine-tuned) if available
if 'history_resnet_ft' in locals() and history_resnet_ft is not None:
    plot_history([history_resnet_ft], ['ResNet Fine-Tune'])

# Plot losses for VGG16 (fine-tuned) if available
if 'history_vgg_ft' in locals() and history_vgg_ft is not None:
    plot_history([history_vgg_ft], ['History VGG Fine-Tune'])

In [ ]:
# ============================================================
# 9. Evaluate and Compare Models
# ============================================================
# Apply preprocessing to test_ds for the custom model evaluation
# Ensure IMG_SIZE is 224 for custom model evaluation
IMG_SIZE_EVAL = 224 # Use a different variable name for clarity

# Explicitly resize X_test before creating the dataset
X_test_resized = tf.image.resize(tf.expand_dims(X_test, -1), [IMG_SIZE_EVAL, IMG_SIZE_EVAL]).numpy()
X_test_rgb = tf.image.grayscale_to_rgb(tf.constant(X_test_resized, dtype=tf.float32)).numpy() / 255.0

# Add print statement to check the shape of X_test_rgb
print("Shape of X_test_rgb before creating dataset:", X_test_rgb.shape)

test_ds_custom = (tf.data.Dataset.from_tensor_slices((X_test_rgb, y_test))
           .batch(BATCH)
           .prefetch(tf.data.AUTOTUNE))


# Add print statement to check the shape of the data from the dataset
for images, labels in test_ds_custom.take(1):
    print("Shape of a batch from test_ds_custom:", images.shape)

test_loss_c, test_acc_c = custom_model.evaluate(test_ds_custom)
print("Custom CNN Test accuracy:", test_acc_c)

test_loss_r, test_acc_r = resnet_model.evaluate(test_ds_resnet)
print("ResNet50 Test accuracy:", test_acc_r)

test_loss_v, test_acc_v = vgg_model.evaluate(test_ds_vgg)
print("VGG16 Test accuracy:", test_acc_v)

summary = pd.DataFrame({
    'Model': ['Custom CNN', 'ResNet50', 'VGG16'],
    'Test Accuracy': [test_acc_c, test_acc_r, test_acc_v]
})
print(summary)

In [ ]:
# ============================================================
# 10. Classification report & confusion matrix (optional)
# ============================================================
# Regenerate y_true based on the correctly preprocessed test_ds for the custom model
y_true_custom = np.concatenate([y for _, y in test_ds_custom], axis=0)

y_pred_c = np.argmax(custom_model.predict(test_ds_custom), axis=1)
print("\nCustom CNN:\n", classification_report(y_true_custom, y_pred_c, digits=4))
cm_c = confusion_matrix(y_true_custom, y_pred_c)
plt.figure(figsize=(8, 6)) # Added figure size for better visualization
sns.heatmap(cm_c, annot=True, fmt='d', cmap='Blues')
plt.title('Custom CNN Confusion Matrix'); plt.show()

# For ResNet50 evaluation
y_true_resnet = np.concatenate([y for _, y in test_ds_resnet], axis=0)
y_pred_r = np.argmax(resnet_model.predict(test_ds_resnet), axis=1)
print("\nResNet50:\n", classification_report(y_true_resnet, y_pred_r, digits=4))
cm_r = confusion_matrix(y_true_resnet, y_pred_r)
plt.figure(figsize=(8, 6)) # Added figure size for better visualization
sns.heatmap(cm_r, annot=True, fmt='d', cmap='Blues')
plt.title('ResNet50 Confusion Matrix'); plt.show()

# For VGG16 evaluation
y_true_vgg = np.concatenate([y for _, y in test_ds_vgg], axis=0)
y_pred_v = np.argmax(vgg_model.predict(test_ds_vgg), axis=1)
print("\nVGG16:\n", classification_report(y_true_vgg, y_pred_v, digits=4))
cm_v = confusion_matrix(y_true_vgg, y_pred_v)
plt.figure(figsize=(8, 6)) # Added figure size for better visualization
sns.heatmap(cm_v, annot=True, fmt='d', cmap='Blues')
plt.title('VGG16 Confusion Matrix'); plt.show()